# Einlesen der log Dateien aus Bordrechnern, extrahieren der Umsteigehinweise

In [ ]:
import pandas as pd
import numpy as np
import duckdb
import os

__Beschreibung Oliver 13.11.2025__

die Beschreibung der Schnittstelle liegt mir vor. So habe ich mein Visualisierungstool gebaut (welches aber nicht die Umsteigehinweise beinhaltet). Die Umsteigehinweise beginnen immer mit K#132#I#EVB/113#24#1.1##320191, wobei 132 das systeminterne Modul ist, welches für die Umsteigehinweise zuständig ist, Fahrzeugnummer und Haltepunkt sind selbsterklärend, die 24#1.1 ist ein Teletype + Version. Trenner ist immer ‚#‘. Das heißt, wenn ‚##‘ auftaucht, ist ein Feld leer.

In [ ]:
file_path = 'NetPeerManager.2025-11-13.1.log' #Original IVU Log Datei
file_path_new = file_path + '.reduced' #Reduzierte Datei mit Umsteigehinweisen Modul 132

## reduzieren des Logs auf Zeilen mit Umsteigehinweisen (Modul 132)

In [ ]:
if os.path.exists(file_path_new):
    os.remove(file_path_new)

with open(file_path, encoding="Latin1") as f:
    for line in f:
        s =line.split('#')
        if len(s) > 2: #kurze Einträge für die weitere Prüfung nicht berücksichtigen
            if s[1] == '132': #Auswahl der Umsteigehinweise Modul 132
                with open(file_path_new, 'a', encoding="Latin1") as f_new:
                    f_new.write(line)

## Auslesen Umsteigehinweise und Parsen

In [ ]:
ln = 0 #laufende Nummer Zeilen
arr_concat = [] #initialisieren der Liste für DataFrames

with open(file_path_new, encoding="Latin1") as f:    
    for line in f:
        ln += 1 #Zeilennummer
        ts = line[0:29]  #Timestamp extrahieren
        s = line.split('#') #Aufspalten der Zeile in Liste 
        fzg = s[3] #Fahrzeugmandant        

        #print(fzg, ln, len(s), s[0][-1:], len(s)%14, s[1])
        arr = np.array(s[6:-3]) #in NumPy Array umwandeln, nur die Datenfelder ohne Felder zu Fzg etc und ohne die letzten 3 Felder (in der Regel leer)
        arr = arr.reshape(-1,14) #Umformen in Array mit 14 Spalten    
        df = pd.DataFrame(arr) #in DataFrame umwandeln
        df['Fzg'] = fzg #Anfügen der Fahrzeugnummer als neue Spalte      
        df['timestamp'] = ts #Anfügen des Timestamps als neue Spalte
        arr_concat.append(df)

### Erstellen eines Dataframes

In [ ]:
result_df_raw = pd.concat(arr_concat, ignore_index=True)
result_df_raw.columns = ['Timestamp1', 'HaltID', 'Halt', 
                     'Mode', 'Unknown1', 'Unknown2', 'Unknown3', 
                     'LinieIntern', 'Linie', 'HaltId2', 'Halt2', 
                     'Unknown4', 'JsonText', 'Timestamp2', 
                     'Fzg', 'timestamp']

In [ ]:
result_df_raw.shape #Größe der DataFrame anzeigen

In [ ]:
result_df_raw.head()

### Ersetzen der leeren Werte bei HaltID und Halt, Umwandlung der Typen

In [ ]:
result_df = result_df_raw.copy()
result_df.replace({"HaltID": "", "Halt":"0"},None, inplace=True) #leere Strings in None umwandeln

In [ ]:
result_df[['HaltID','Halt']] = result_df[['HaltID','Halt']].ffill(inplace=False) #Forward fill auf Spalten Halt und HaltIDanwenden

In [ ]:
result_df['HaltID'] = result_df['HaltID'].astype('Int64')

In [ ]:
result_df.head()

In [ ]:
result_df['timestamp'] = pd.to_datetime(result_df['timestamp'], errors='raise',format="%Y-%m-%dT%H:%M:%S,%f%z") #in Zeittyp umwandeln

In [ ]:
result_df.head()

### Auswertung der Anzahl nach Linien und Fahrzeugen Dataframe

In [ ]:
# Concatenate all DataFrames in arr_concat into a single DataFrame
result_df['Linie'].value_counts()

In [ ]:
# Concatenate all DataFrames in arr_concat into a single DataFrame
result_df['Fzg'].value_counts()

### Auswertung für Zeitbereich Dataframe

In [ ]:
result_df.timestamp.min(), result_df.timestamp.max()

In [ ]:
zeit_fil = '2025-11-13 05:45:00+0100'
df_zeit = result_df[result_df['timestamp'] >= zeit_fil]
df_zeit

### Ausgabe als Parquet

In [ ]:
file_path_parquet = file_path + '.parquet'

In [ ]:
result_df.to_parquet(os.path.join('out', file_path_parquet), index=False)

## Auswertung in DuckDB

- später als getrenntes Modul

In [ ]:
con = duckdb.connect(database=':memory:')
#einelsen aller parquet aus Ordner
con.execute("CREATE TABLE umstiege AS SELECT * FROM read_parquet('out/*2025-11-13*.parquet');")
con.sql("select count(*) from umstiege")

### Übermittelte Linien je Fzg 

- Ausgabe von Kennwerten je Fahrzeug
- Ermitteln der externen Anschlüsse

In [ ]:
weitere_regio = "'S35','3860'"

In [ ]:
con.sql(f"""
    SELECT distinct linie, case when (not REGEXP_MATCHES(linie, '^[0-9]{{3}}$')) and (linie not in ({weitere_regio}))  then linie end as linie_ohne_regio
        from umstiege
        -- Beschränkung auf bestimmte Fahrzeuge / Ausschluss LSE Fahrzeuge
        where fzg not like '%LSE%' 
        order by linie_ohne_regio, linie""").df()

In [ ]:
con.sql(f"""
        select distinct fzg, min(timestamp) as min_ts, max(timestamp) as max_ts, count(*) as anzahl,
        count(distinct linie) as linienanzahl,        
        string_agg(distinct linie, ', ' ORDER BY linie) as linien,
        string_agg(distinct linie_ohne_regio, ', ' ORDER BY linie_ohne_regio) as linien_ohne_regio
        
        from 
            -- Tabelle mit zusätzlicher Spalte linie_ohne_regio nicht dreistellige Liniennummern und weitere Regio Linien
            (SELECT *, 
            case when (not REGEXP_MATCHES(linie, '^[0-9]{{3}}$')) and (linie not in ({weitere_regio}))  then linie end as linie_ohne_regio
            from umstiege) 
                
        -- Beschränkung auf bestimmte Fahrzeuge / Ausschluss LSE Fahrzeuge
        where fzg not like '%LSE%' 
        
        group by fzg
        order by fzg""").df()

In [ ]:
con.close()